In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
import seaborn as sns

%matplotlib inline
#%matplotlib notebook
%reload_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = 10, 5

# Range-only datasets for localization 

### Download .mat files and save them in folder ./datasets/

WiFi: http://www.robesafe.es/repository/UAHWiFiDataset/

Landmower: https://github.com/gtrll/gpslam/tree/master/matlab/data


File description (from WiFi website, seems to be similar for Landmower): 
```
GT: Groundtruth path from Laser Scan matching
Time (sec) 	X_pose (m) 	Y_pose (m) 	Heading (rad)

DR: Odometry Input (delta distance traveled and delta heading change)
Time (sec) 	Delta Dist. Trav. (m) 	Delta Heading (rad)

DR_PROC: Dead Reckoned Path from Odometry
Time (sec) 	X_pose (m) 	Y_pose (m) 	Heading (rad)

TL: Surveyed Beacon Locations
Time (sec) 	X_pose (m) 	Y_pose (m)
*NOTE by Frederike: above is probably ID instead of time.

TD:
Time (sec) 	Sender / Antenna ID 	Receiver Node ID 	Range (m)
```

In [ ]:
from trajectory_creator import get_trajectory

# Need to give different systems a name.
gt_system_id = "GT"
gt_anchor_id = "GT"
range_system_id = "Range"

t_window = 0.1 # Time window to use for ground truth interpolation (see add_gt_raw).

#filename = 'datasets/uah1.mat'; # fingers. does not work well, did not realy try.
#min_time = 0
#max_time = 230

filename = 'datasets/uah2.mat'; # square. does not work well.
min_time = 0
max_time = 1470
t_window = 1.0

#filename = 'datasets/Plaza1.mat'; # zig zag. does not work super well.
#min_time = 0 # first big circle
#max_time = 200 # first big circle
#min_time=510 # first loop
#max_time=600 # first loop

#filename = 'datasets/Plaza2.mat' # triangle. works well.
#max_time=100.3
#min_time=45.1

traj = get_trajectory(filename)

try: 
    result_dict = loadmat(filename)
    print('read', filename)
except FileNotFoundError:
    print('ERROR: download the mat files as described in cell above.')

In [ ]:
from evaluate_dataset import format_anchors_df, format_data_df

anchor_data = result_dict['TL']
anchors_df = pd.DataFrame(columns=['anchor_id', 'system_id', 'px', 'py', 'pz'])
anchor_ids = np.unique(anchor_data[:, 0])
for i, anchor_id in enumerate(anchor_ids):
    anchors_df.loc[i, 'anchor_id'] = anchor_id 
    anchors_df.loc[i, 'system_id'] = range_system_id
    
    # it is weird that there is more than one value for each anchor, it looks
    # like this was a bug in the dataset. we make sure they are all
    # the same and pick the first.
    px_values = np.unique(anchor_data[anchor_data[:, 0]==anchor_id, 1])
    py_values = np.unique(anchor_data[anchor_data[:, 0]==anchor_id, 2])
    assert len(px_values) == 1
    assert len(py_values) == 1
    anchors_df.loc[i, 'px'] = px_values[0]
    anchors_df.loc[i, 'py'] = py_values[0]
    
anchors_df = format_anchors_df(anchors_df, range_system_id=range_system_id, 
                               gt_system_id=gt_system_id)
print('anchors: \n', anchors_df)

range_df = pd.DataFrame(columns=['timestamp', 'px', 'py', 'pz', 'distance', 
                            'system_id', 'anchor_id'])
range_df.loc[:, 'distance'] = result_dict['TD'][:, 3]
range_df.loc[:, 'timestamp'] = result_dict['TD'][:, 0]
range_df.loc[:, 'anchor_id'] = result_dict['TD'][:, 2]
range_df.loc[:, 'system_id'] = range_system_id 

gt_df = pd.DataFrame(columns=range_df.columns)
gt_df.loc[:, 'px'] = result_dict['GT'][:, 1]
gt_df.loc[:, 'py'] = result_dict['GT'][:, 2]
gt_df.loc[:, 'timestamp'] = result_dict['GT'][:, 0]
gt_df.loc[:, 'anchor_id'] = gt_anchor_id
gt_df.loc[:, 'system_id'] = gt_system_id

full_df = pd.concat([range_df, gt_df], ignore_index=True)
full_df.sort_values('timestamp', inplace=True)
full_df.reset_index(drop=True, inplace=True)
full_df.loc[:, 'timestamp'] = full_df.timestamp -full_df.timestamp.min()
print('time going from {:.1f} to {:.1f}'.format(full_df.timestamp.min(), full_df.timestamp.max()))
new_df = full_df[(full_df.timestamp >= min_time) & (full_df.timestamp <= max_time)]
print('warning: keeping {}/{}'.format(len(new_df), len(full_df)))
full_df = new_df
full_df = format_data_df(full_df, anchors_df, gt_system_id=gt_system_id, 
                         range_system_id=range_system_id)
full_df.loc[:, 'timestamp'] = full_df.timestamp - full_df.timestamp.min()

In [ ]:
%matplotlib notebook
fig, axs = plt.subplots(1, 2)
sns.scatterplot(data=full_df, x='px', y='py', hue='timestamp', linewidth=0.0, 
                ax=axs[0])
sns.scatterplot(data=full_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, 
                 ax=axs[1])

In [ ]:
from evaluate_dataset import add_gt_raw
full_df = add_gt_raw(full_df, t_window=t_window, gt_system_id=gt_system_id)

In [ ]:
fig, axs = plt.subplots(1, 2)
range_df = full_df[full_df.system_id==range_system_id]
sns.scatterplot(data=range_df, x='px', y='py', hue='timestamp', linewidth=0.0, 
                ax=axs[0])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')
sns.scatterplot(data=range_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, 
                 ax=axs[1])

In [ ]:
from evaluate_dataset import apply_distance_gt
full_df.loc[:, "distance_gt"] = full_df.apply(lambda row: apply_distance_gt(row, anchors_df, gt_system_id=gt_system_id), axis=1)

In [ ]:
range_ids = full_df[full_df.system_id==range_system_id].anchor_id.unique()
fig, axs = plt.subplots(len(range_ids), sharex=True)
fig.set_size_inches(10, 10)
for i, anchor_id in enumerate(sorted(range_ids)):
    this_df = full_df[full_df.anchor_id==anchor_id]
    axs[i].scatter(this_df.timestamp, this_df.distance, color='red', label='measured distance')
    axs[i].scatter(this_df.timestamp, this_df.distance_gt, color='green', label='real distance')
    axs[i].legend(loc='upper right')
    axs[i].set_title('anchor {}'.format(anchor_id))
    axs[i].set_ylabel('distance [m]')
axs[i].set_xlabel('time [s]')

# Construct D

In [ ]:
from evaluate_dataset import get_length, compute_distance_matrix
from trajectory import Trajectory

# We want to find out the times of distance measurements, 
# and by how much we have moved.
range_df = full_df[full_df.system_id==range_system_id]
times = range_df.timestamp.unique()
chosen_distance = 'distance'
anchor_names = None # use all anchors.

## Sometimes below changes the length of times and then things seem to crash. Need to find out why.
D, times = compute_distance_matrix(full_df, anchors_df, 
                            anchor_names, times, chosen_distance)
if np.sum(D>0) > D.shape[0]:
    print('Warning: multiple measurements for some times!')
#plt.matshow(D[:10, :])
#plt.title('First 10 rows of D matrix')

# Find corresponding ground truth 

In [ ]:
# Find one ground truth for each time when we have a distance
# measurement. 
ground_truth_pos = full_df.loc[full_df.timestamp.isin(times), ['timestamp', 'px', 'py', 'pz']]
ground_truth_pos = ground_truth_pos.astype(np.float32)
ground_truth_pos = ground_truth_pos.groupby('timestamp').agg(np.nanmean)
ground_truth_pos.reset_index(inplace=True)
ground_truth_pos.fillna(method='ffill', inplace=True)

# We need to translate the times to "trajectory space"
lengths = get_length(ground_truth_pos)
lengths[np.isnan(lengths)] = 0 # because beginning of lengths can still have nans.
assert len(lengths) == D.shape[0]

# Use only distances for which we have valid ground truth.
mask = list(lengths>0) # keep first zero length but delete others.
mask[0] = True
D = D[mask, :]
times = np.array(times)[mask]
lengths = lengths[mask]

time_diffs = times[1:] - times[:-1]
velocities = lengths[1:] / time_diffs
plt.figure()
plt.hist(velocities, bins=20)
plt.title('velocity histogram')

distances = np.cumsum(lengths)

if anchor_names is None:
    anchors = anchors_df.loc[:, ['px', 'py', 'pz']].values.astype(np.float32).T
else:
    raise NotImplementedError('need to implement this (not a big deal).')

In [ ]:
from solvers import alternativePseudoInverse
from plotting_tools import savefig

dataname = filename.split('/')[-1].split('.')[0]

list_complexities = [3, 5, 11, 21, 51]
for n_complexity in list_complexities:
    traj.set_n_complexity(n_complexity)
    
    ###### IMPORTANT: of course, this does not work, because we  ######
    # do not have the "exact" trajectory model. the best we can do ####
    # is to use the actual times and hope they fit the model. I was ###
    # dumb and it took me quite some time figuring this out. ##########
    times_corr,*_ = traj.get_times_from_distances(arbitrary_distances=distances, 
                                                  time_steps=10000)
    ###################################################################
    
    times_corr = times 
    basis = traj.get_basis(times=times_corr) 

    Chat_weighted = alternativePseudoInverse(D, anchors[:2, :], basis, weighted=True)
    Chat = alternativePseudoInverse(D, anchors[:2, :], basis, weighted=False)
    
    traj_weighted = traj.copy()
    
    traj.set_coeffs(coeffs=Chat)
    traj_weighted.set_coeffs(coeffs=Chat_weighted)
    
    ax = traj.plot(times=times_corr, color='green', label='non-weighted')
    traj_weighted.plot(times=times_corr, color='blue', label='weighted', ax=ax)
    ax.plot(full_df.px, full_df.py, color='black', label='ground truth')
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_title('K={}'.format(traj.n_complexity))
    ax.legend()
    #ax.set_xlim(full_df.px.min(), full_df.px.max())
    #ax.set_ylim(full_df.py.min(), full_df.py.max())
    savefig(fig, 'results/{}/K{}.png'.format(dataname, n_complexity))